# --- Day 8: Playground ---
https://adventofcode.com/2025/day/8

## Parse the Input Data

In [1]:
import numpy as np

In [2]:
def parse(filename):
    """Parse puzzle input data."""
    boxes = {}
    with open(f'../inputs/{filename}.txt') as f:
        for line in f:
            k = line.strip()
            v = np.array(tuple(map(int, line.strip().split(','))))
            boxes[k] = v
    return boxes

In [3]:
parse("day-08_test")

{'162,817,812': array([162, 817, 812]),
 '57,618,57': array([ 57, 618,  57]),
 '906,360,560': array([906, 360, 560]),
 '592,479,940': array([592, 479, 940]),
 '352,342,300': array([352, 342, 300]),
 '466,668,158': array([466, 668, 158]),
 '542,29,236': array([542,  29, 236]),
 '431,825,988': array([431, 825, 988]),
 '739,650,466': array([739, 650, 466]),
 '52,470,668': array([ 52, 470, 668]),
 '216,146,977': array([216, 146, 977]),
 '819,987,18': array([819, 987,  18]),
 '117,168,530': array([117, 168, 530]),
 '805,96,715': array([805,  96, 715]),
 '346,949,466': array([346, 949, 466]),
 '970,615,88': array([970, 615,  88]),
 '941,993,340': array([941, 993, 340]),
 '862,61,35': array([862,  61,  35]),
 '984,92,344': array([984,  92, 344]),
 '425,690,689': array([425, 690, 689])}

## Part 1
---

In [4]:
import heapq
from itertools import combinations
from math import prod
from scipy.spatial.distance import pdist

In [5]:
def make_circuts(pairs):
    while True:
        intersections = False

        for i, j in combinations(range(len(pairs)), 2):
            if pairs[j] and pairs[i] & pairs[j]:
                intersections = True
                pairs[i].update(pairs[j])
                pairs[j].clear()

        if not intersections:
            return pairs

In [6]:
list(zip(*np.triu_indices(5, k=1)))

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 3),
 (2, 4),
 (3, 4)]

In [7]:
def solve(coords, n, t):
    dist_dict = {}
    pq = []

    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html
    # https://numpy.org/doc/stable/reference/generated/numpy.triu_indices.html
    distances = pdist(np.array(list(coords.values())))
    indicies = zip(*np.triu_indices(len(coords), k=1))
    coords_keys = list(coords.keys())

    for d, (i, j) in zip(distances, indicies):
        heapq.heappush(pq, d)
        dist_dict[d] = set([coords_keys[i], coords_keys[j]])

    min_keys = heapq.nsmallest(n, pq)
    min_n_pairs = [dist_dict[k] for k in min_keys]

    circuts = make_circuts(min_n_pairs)
    circut_lens = sorted([len(c) for c in circuts], reverse=True)

    return prod(circut_lens[:t])

### Run on Test Data

In [8]:
solve(parse("day-08_test"), 10, 3) == 40

True

### Run on Input Data

In [9]:
solve(parse("day-08"), 1000, 3)

127551

## Part 2
---

In [15]:
def make_circuts2(pairs):
    circuts = []
    while True:
        any_matches = False

        for i, j in combinations(range(len(pairs)), 2):
            if pairs[i] != set() and any(p in pairs[i] for p in pairs[j]):
                any_matches = True
                pairs[j].update(pairs[i])
                pairs[i] = set()

                # print(pairs)

            if len([p for p in pairs if p != set()]) == 2:
                return [p for p in pairs if p]

In [18]:
def solve2(coords):

    dist_dict = {}
    pq = []

    distances = pdist(np.array(list(coords.values())))
    indicies = zip(*np.triu_indices(len(coords), k=1))
    coords_keys = list(coords.keys())

    for d, (i, j) in zip(distances, indicies):
        heapq.heappush(pq, d)
        dist_dict[d] = set([coords_keys[i], coords_keys[j]])

    min_keys = heapq.nsmallest(len(pq), pq)
    min_n_pairs = [dist_dict[k] for k in min_keys]
    print(min_n_pairs)
    return
    circuts = make_circuts2(min_n_pairs)
    print(circuts)

### Run on Test Data

In [ ]:
solve2(parse("day-08_test")) == 25272

### Run on Input Data

In [ ]:
solve2(parse("day-08"))